### __Задача 1__:

Реализуйте 2 функции препроцессинга:

- Удалить именованные сущности с помощью natasha (https://github.com/natasha/yargy)
- Удалить именованные сущности с помощью deepmipt (https://github.com/deepmipt/ner)


# NATASHA

In [ ]:
!pip install --quiet natasha

In [ ]:
import natasha
import pandas as pd

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsNERTagger,
    
    PER,
    DatesExtractor,
    NamesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

answers = pd.read_excel('/content/drive/My Drive/PROGA/infosearch/hw2/answers_base.xlsx')
queries = pd.read_excel("/content/drive/My Drive/PROGA/infosearch/hw2/queries_base.xlsx").drop(columns = ['Unnamed: 3', 'Unnamed: 4'])

In [ ]:
answers.head(3)

,Номер связки,Текст вопросов,Текст ответа,Тематика
0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ


In [ ]:
queries.head(3)

,Текст вопроса,Номер связки\n,Тематика
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА..."
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ


In [ ]:
morph_vocab = MorphVocab()
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
ner_tagger = NewsNERTagger(emb)
dates_extractor = DatesExtractor(morph_vocab)
names_extractir = NamesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

# не знаю, стоит ли делать dates_extractor, потому что , во-первых, это есть в задании 3, 
# а в-вторых, наташа выделяет только даты, если это убрать, то останутся слов месяц и год, например:
# После августа  года к нам, в Москву планируют приехать гости...

# a AddrExtractor выделяет города, а вот улицы нет ((

In [ ]:
from tqdm.auto import tqdm

def preprocess_with_natasha(text: str) -> str:
    
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    for x in doc.spans:
        text = text.replace(x.text,'')
    
    return text


texts_no_ner = []
texts = list(queries['Текст вопроса'])
for text in tqdm(texts):
    if type(text) == str:
        texts_no_ner.append(preprocess_with_natasha(text))
    else:
        texts_no_ner.append(text)

In [ ]:
queries['Текст вопроса без NER NATASHA'] = texts_no_ner
queries.head(3)

,Текст вопроса,Номер связки\n,Тематика,Текст вопроса без NER NATASHA
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",с уважением \n\nПосле 15 августа 2020 года к н...
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,"Здравствуйте! Проинформируйте, пожалуйста, нуж..."
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,"--\nДобрый день!\n Меня, 30.07.1989г.р., п..."


In [ ]:
texts_no_ner = []
texts = list(answers['Текст вопросов'])
for text in tqdm(texts):
    if type(text) == str:
        texts_no_ner.append(preprocess_with_natasha(text))
    else:
        texts_no_ner.append(text)
        
answers['Текст вопросов без NER NATASHA'] = texts_no_ner
answers.head(3)        

,Номер связки,Текст вопросов,Текст ответа,Тематика,Текст вопросов без NER DEEPMINT,Текст вопроса без NER DEEPMINT,Текст вопросов без NER NATASHA
0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы. Могу ли я...
1,78,Где сделать вакцинацию от коронавируса?\nСущес...,"Коронавирусы - это целое семейство вирусов, ко...",ВАКЦИНАЦИЯ,Где сделать вакцинацию от коронавируса?\nСущес...,Где сделать вакцинацию от коронавируса?\nСущес...,Где сделать вакцинацию от коронавируса?\nСущес...
2,326,Сколько стоит сделать вакцину от гриппа?\nМожн...,Бесплатно пройти вакцинацию можно в Вашей меди...,ВАКЦИНАЦИЯ,Сколько стоит сделать вакцину от гриппа?\nМожн...,Сколько стоит сделать вакцину от гриппа?\nМожн...,Сколько стоит сделать вакцину от гриппа?\nМожн...


# DEEPMINT

In [ ]:
!pip install deeppavlov
!python -m deeppavlov install ner_conll2003_bert

from deeppavlov import configs, build_model

In [ ]:
model = build_model(configs.ner.ner_rus_bert, download=True)

In [ ]:
for text in tqdm(texts[:1]):
  result = model([text])
  words = result[0][0]
  tags = result[1][0]
  for i, el in enumerate(tags):
    if el !=  'O':
      text = text.replace(words[i], '')


'с уважением   \n\nПосле 15 августа 2020 года к нам, в  планируют приехать гости из .\nКакой на сегодняшний день режим по  распространяется на иностранцев, прибывающих в ?\n\nС уважением,\n \n'

In [ ]:
# bert падает, если токенов > 512, поэтому будем делить на предложения
from razdel import sentenize

def preprocess_with_deepmint(text: str) -> str:
  substrings =  sentenize(text)
  sentences = [_.text for _ in substrings]
  for sent in sentences:
    result = model([sent])
    words = result[0][0]
    tags = result[1][0]
    for i, el in enumerate(tags):
      if el !=  'O':
        text = text.replace(words[i], '')

  return text


texts_no_ner = []
texts = list(queries['Текст вопроса'])
for text in tqdm(texts):
    if type(text) == str:
        texts_no_ner.append(preprocess_with_deepmint(text))
    else:
        texts_no_ner.append(text)

queries['Текст вопроса без NER DEEPMINT'] = texts_no_ner
queries.head(3)

,Текст вопроса,Номер связки\n,Тематика,Текст вопроса без NER NATASHA,Текст вопроса без NER DEEPMINT
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",с уважением \n\nПосле 15 августа 2020 года к н...,с уважением \n\nПосле 15 августа 2020 года ...
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,"Здравствуйте! Проинформируйте, пожалуйста, нуж...","Здравствуйте! Проинформируйте, пожалуйста, нуж..."
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,"--\nДобрый день!\n Меня, 30.07.1989г.р., п...","--\nДобрый день!\n Меня, 30.07.1989г.р., ..."


In [ ]:
texts_no_ner = []
texts = list(answers['Текст вопросов'])
for text in tqdm(texts):
    if type(text) == str:
        texts_no_ner.append(preprocess_with_deepmint(text))
    else:
        texts_no_ner.append(text)

answers['Текст вопроса без NER DEEPMINT'] = texts_no_ner
answers.head(3)

In [ ]:
queries.to_excel('queries_ner.xlsx')
answers.to_excel('answers_ner.xlsx')